## 1. Data Acquisition 

In [6]:
import json
import pandas as pd
from typing import Any, Dict, List
import datetime

def process_nvd_json(file_path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Process an NVD JSON file and convert it to two normalized DataFrames:
    1. Main CVE DataFrame
    2. CPE matches DataFrame with foreign key to CVE
    
    Args:
        file_path (str): Path to the NVD JSON file
        
    Returns:
        tuple[pd.DataFrame, pd.DataFrame]: Tuple containing (cve_df, cpe_df)
    """
    # Read the JSON file
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        nvd_data = json.load(f)
    
    # Lists to store processed items
    cve_items = []
    cpe_items = []
    
    for cve_item in nvd_data['CVE_Items']:
        cve_data = {}
        
        # Basic CVE information
        cve_id = cve_item['cve']['CVE_data_meta']['ID']
        cve_data['id'] = cve_id
        cve_data['assigner'] = cve_item['cve']['CVE_data_meta'].get('ASSIGNER', '')
        cve_data['published_date'] = cve_item.get('publishedDate', None)
        cve_data['last_modified_date'] = cve_item.get('lastModifiedDate', None)
        
        # Description
        descriptions = cve_item['cve']['description']['description_data']
        cve_data['description'] = next((desc['value'] for desc in descriptions if desc.get('lang') == 'en'), '')
        
        # Problem type (CWE)
        try:
            problemtype_data = cve_item['cve']['problemtype']['problemtype_data']
            if problemtype_data and problemtype_data[0].get('description'):
                cve_data['cwe'] = problemtype_data[0]['description'][0].get('value', '')
            else:
                cve_data['cwe'] = ''
        except (KeyError, IndexError):
            cve_data['cwe'] = ''
        
        # References
        try:
            references = cve_item['cve']['references']['reference_data']
            cve_data['references'] = '; '.join(ref.get('url', '') for ref in references)
        except (KeyError, IndexError):
            cve_data['references'] = ''
        
        # CVSS v3 metrics
        try:
            cvss3 = cve_item['impact']['baseMetricV3']['cvssV3']
            cve_data['cvss3_vector'] = cvss3.get('vectorString', '')
            cve_data['cvss3_base_score'] = cvss3.get('baseScore', None)
            cve_data['cvss3_base_severity'] = cvss3.get('baseSeverity', '')
        except (KeyError, TypeError):
            cve_data.update({
                'cvss3_vector': '',
                'cvss3_base_score': None,
                'cvss3_base_severity': ''
            })
        
        # Process CPE matches
        try:
            nodes = cve_item['configurations']['nodes']
            for node in nodes:
                for cpe in node.get('cpe_match', []):
                    cpe_info = {
                        'cve_id': cve_id,
                        'cpe23Uri': cpe.get('cpe23Uri', ''),
                        'vulnerable': cpe.get('vulnerable', False),
                        'versionStartIncluding': cpe.get('versionStartIncluding', ''),
                        'versionEndIncluding': cpe.get('versionEndIncluding', '')
                    }
                    cpe_parts = cpe_info['cpe23Uri'].split(':')
                    if len(cpe_parts) > 4:
                        cpe_info.update({
                            'vendor': cpe_parts[3],
                            'product': cpe_parts[4],
                            'version': cpe_parts[5]
                        })
                    cpe_items.append(cpe_info)
        except (KeyError, TypeError):
            pass
        
        cve_items.append(cve_data)
    
    # Create DataFrames
    cve_df = pd.DataFrame(cve_items)
    cpe_df = pd.DataFrame(cpe_items)
    
    # Convert date columns to datetime
    for col in ['published_date', 'last_modified_date']:
        cve_df[col] = pd.to_datetime(cve_df[col], errors='coerce')
    
    # Convert CVSS score to numeric
    cve_df['cvss3_base_score'] = pd.to_numeric(cve_df['cvss3_base_score'], errors='coerce')
    
    # Sort DataFrames
    cve_df = cve_df.sort_values('id')
    cpe_df = cpe_df.sort_values(['cve_id', 'cpe23Uri'])
    return cve_df, cpe_df

## 2. NVD Data Loading and Initial Processing 

In [9]:
# The vulnerbilities for 2024 in the NVD
cve_df, cpe_df = process_nvd_json('nvdcve-1.1-2024.json')

# Display info & head
cve_df.info()
cve_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37730 entries, 0 to 37729
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   id                   37730 non-null  object             
 1   assigner             37730 non-null  object             
 2   published_date       37730 non-null  datetime64[ns, UTC]
 3   last_modified_date   37730 non-null  datetime64[ns, UTC]
 4   description          37730 non-null  object             
 5   cwe                  37730 non-null  object             
 6   references           37730 non-null  object             
 7   cvss3_vector         37730 non-null  object             
 8   cvss3_base_score     20271 non-null  float64            
 9   cvss3_base_severity  37730 non-null  object             
dtypes: datetime64[ns, UTC](2), float64(1), object(7)
memory usage: 2.9+ MB


,id,assigner,published_date,last_modified_date,description,cwe,references,cvss3_vector,cvss3_base_score,cvss3_base_severity
0,CVE-2024-0001,psirt@purestorage.com,2024-09-23 18:15:00+00:00,2024-09-27 14:08:00+00:00,A condition exists in FlashArray Purity whereb...,CWE-1188,https://purestorage.com/security,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,9.8,CRITICAL
1,CVE-2024-0002,psirt@purestorage.com,2024-09-23 18:15:00+00:00,2024-09-27 14:13:00+00:00,A condition exists in FlashArray Purity whereb...,NVD-CWE-noinfo,https://purestorage.com/security,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,9.8,CRITICAL
2,CVE-2024-0003,psirt@purestorage.com,2024-09-23 18:15:00+00:00,2024-09-27 14:23:00+00:00,A condition exists in FlashArray Purity whereb...,NVD-CWE-noinfo,https://purestorage.com/security,CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:H/I:H/A:H,7.2,HIGH
3,CVE-2024-0004,psirt@purestorage.com,2024-09-23 18:15:00+00:00,2024-09-27 14:24:00+00:00,A condition exists in FlashArray Purity whereb...,CWE-94,https://purestorage.com/security,CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:H/I:H/A:H,7.2,HIGH
4,CVE-2024-0005,psirt@purestorage.com,2024-09-23 18:15:00+00:00,2024-09-27 15:25:00+00:00,A condition exists in FlashArray and FlashBlad...,CWE-77,https://purestorage.com/security,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,8.8,HIGH


## 3. KEV Data Loading

In [11]:
kev_df = pd.read_csv('known_exploited_vulnerabilities.csv')

# Renaming the columns for clarity
kev_df.rename(columns={'cveID':'id', 'dateAdded':'kev_added_date'}, inplace=True)
#Coverts to a data time object
kev_df['kev_added_date'] = pd.to_datetime(kev_df['kev_added_date'], errors='coerce')

# Display info & the first 5 rows
print('KEV DataFrame info:')
kev_df.info()
kev_df.head()

KEV DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          1323 non-null   object        
 1   vendorProject               1323 non-null   object        
 2   product                     1323 non-null   object        
 3   vulnerabilityName           1323 non-null   object        
 4   kev_added_date              1323 non-null   datetime64[ns]
 5   shortDescription            1323 non-null   object        
 6   requiredAction              1323 non-null   object        
 7   dueDate                     1323 non-null   object        
 8   knownRansomwareCampaignUse  1323 non-null   object        
 9   notes                       1323 non-null   object        
 10  cwes                        1177 non-null   object        
dtypes: datetime64[ns](1), object(10)
mem

,id,vendorProject,product,vulnerabilityName,kev_added_date,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,notes,cwes
0,CVE-2025-24054,Microsoft,Windows,Microsoft Windows NTLM Hash Disclosure Spoofin...,2025-04-17,Microsoft Windows NTLM contains an external co...,"Apply mitigations per vendor instructions, fol...",2025-05-08,Unknown,https://msrc.microsoft.com/update-guide/vulner...,CWE-73
1,CVE-2025-31201,Apple,Multiple Products,Apple Multiple Products Arbitrary Read and Wri...,2025-04-17,"Apple iOS, iPadOS, macOS, and other Apple prod...","Apply mitigations per vendor instructions, fol...",2025-05-08,Unknown,https://support.apple.com/en-us/122282 ; https...,NaN
2,CVE-2025-31200,Apple,Multiple Products,Apple Multiple Products Memory Corruption Vuln...,2025-04-17,"Apple iOS, iPadOS, macOS, and other Apple prod...","Apply mitigations per vendor instructions, fol...",2025-05-08,Unknown,https://support.apple.com/en-us/122282 ; https...,NaN
3,CVE-2021-20035,SonicWall,SMA100 Appliances,SonicWall SMA100 Appliances OS Command Injecti...,2025-04-16,SonicWall SMA100 appliances contain an OS comm...,"Apply mitigations per vendor instructions, fol...",2025-05-07,Unknown,https://psirt.global.sonicwall.com/vuln-detail...,CWE-78
4,CVE-2024-53150,Linux,Kernel,Linux Kernel Out-of-Bounds Read Vulnerability,2025-04-09,Linux Kernel contains an out-of-bounds read vu...,"Apply mitigations per vendor instructions, fol...",2025-04-30,Unknown,This vulnerability affects a common open-sourc...,CWE-125


## 4. Merging Datasets

In [13]:
# Merge NVD and KEV
merged_df = cve_df.merge(kev_df, on='id', how='left')

#Checks if the vulnerability exists in the merged data
merged_df['is_exploited'] = merged_df['kev_added_date'].notna()


## 5. Output and Verification

In [15]:
# Display the merged info and first 5 rows
print('Merged DataFrame info:')
merged_df.info()
print('\nFirst 5 rows:')
merged_df.head()

#Displays the counts of exploited values
print('\nExploited value counts:')
merged_df['is_exploited'].value_counts()

Merged DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37730 entries, 0 to 37729
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   id                          37730 non-null  object             
 1   assigner                    37730 non-null  object             
 2   published_date              37730 non-null  datetime64[ns, UTC]
 3   last_modified_date          37730 non-null  datetime64[ns, UTC]
 4   description                 37730 non-null  object             
 5   cwe                         37730 non-null  object             
 6   references                  37730 non-null  object             
 7   cvss3_vector                37730 non-null  object             
 8   cvss3_base_score            20271 non-null  float64            
 9   cvss3_base_severity         37730 non-null  object             
 10  vendorProject               142 non

is_exploited
False    37588
True       142
Name: count, dtype: int64

In [36]:
merged_df.to_csv(
    'merged_full_nvd_kev_2024.csv',
    index=False)